In [31]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


file_path = 'extracted_features-2.csv' 
data = pd.read_csv(file_path)
data['trigger'] = data['trigger'].map({-1: 0, 1: 1})
train_data = data[data['participant_id'] != 5]
test_data = data[data['participant_id'] == 5]


X_train = train_data.drop(columns=['trigger', 'participant_id']).values
y_train = train_data['trigger'].values

X_test = test_data.drop(columns=['trigger', 'participant_id']).values
y_test = test_data['trigger'].values


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1) 
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1) 

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size).to(x.device) 
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  
        return out.squeeze()

input_size = X_train_tensor.shape[2] 
hidden_size = 64

model = SimpleRNN(input_size, hidden_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100 
for epoch in range(epochs):
    model.train()
    epoch_losses = []
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    avg_loss = sum(epoch_losses) / len(epoch_losses)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')

model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        predicted = (torch.sigmoid(output) >= 0.5).int()
        y_pred.extend(predicted.tolist())
        y_true.extend(y_batch.int().tolist())

y_true_int = np.array(y_true).astype(int)
y_pred_int = np.array(y_pred).astype(int)

accuracy = accuracy_score(y_true_int, y_pred_int)
precision = precision_score(y_true_int, y_pred_int, zero_division=0)
recall = recall_score(y_true_int, y_pred_int, zero_division=0)
f1 = f1_score(y_true_int, y_pred_int, zero_division=0)

print(f'RNN Model - Accuracy: {accuracy:.4f}')
print(f'RNN Model - Precision: {precision:.4f}')
print(f'RNN Model - Recall: {recall:.4f}')
print(f'RNN Model - F1 Score: {f1:.4f}')


Epoch 1/100, Loss: 0.4760
Epoch 2/100, Loss: 0.3651
Epoch 3/100, Loss: 0.3567
Epoch 4/100, Loss: 0.3522
Epoch 5/100, Loss: 0.3466
Epoch 6/100, Loss: 0.3436
Epoch 7/100, Loss: 0.3400
Epoch 8/100, Loss: 0.3363
Epoch 9/100, Loss: 0.3307
Epoch 10/100, Loss: 0.3267
Epoch 11/100, Loss: 0.3217
Epoch 12/100, Loss: 0.3179
Epoch 13/100, Loss: 0.3124
Epoch 14/100, Loss: 0.3085
Epoch 15/100, Loss: 0.3041
Epoch 16/100, Loss: 0.3000
Epoch 17/100, Loss: 0.2931
Epoch 18/100, Loss: 0.2896
Epoch 19/100, Loss: 0.2841
Epoch 20/100, Loss: 0.2775
Epoch 21/100, Loss: 0.2751
Epoch 22/100, Loss: 0.2691
Epoch 23/100, Loss: 0.2634
Epoch 24/100, Loss: 0.2587
Epoch 25/100, Loss: 0.2541
Epoch 26/100, Loss: 0.2490
Epoch 27/100, Loss: 0.2426
Epoch 28/100, Loss: 0.2397
Epoch 29/100, Loss: 0.2324
Epoch 30/100, Loss: 0.2285
Epoch 31/100, Loss: 0.2252
Epoch 32/100, Loss: 0.2157
Epoch 33/100, Loss: 0.2142
Epoch 34/100, Loss: 0.2077
Epoch 35/100, Loss: 0.2035
Epoch 36/100, Loss: 0.1980
Epoch 37/100, Loss: 0.1928
Epoch 38/1

In [32]:
file_path = 'extracted_features-2.csv' 
data = pd.read_csv(file_path)

data['trigger'] = data['trigger'].map({-1: 0, 1: 1})

train_data = data[data['participant_id'] != 5]
test_data = data[data['participant_id'] == 5]

X_train = train_data.drop(columns=['trigger', 'participant_id']).values
y_train = train_data['trigger'].values

X_test = test_data.drop(columns=['trigger', 'participant_id']).values
y_test = test_data['trigger'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1) 
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_prob):
        super(BiLSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_dim, hidden_dim1, batch_first=True, bidirectional=True)
        self.dropout1 = nn.Dropout(dropout_prob)
        self.lstm2 = nn.LSTM(hidden_dim1 * 2, hidden_dim2, batch_first=True, bidirectional=True)
        self.dropout2 = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_dim2 * 2, output_dim)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.dropout2(x)
        x = self.fc(x[:, -1, :])
        return x.squeeze()


input_dim = X_train_tensor.shape[2] 
hidden_dim1 = 10
hidden_dim2 = 5
output_dim = 1
dropout_prob = 0.2

model_BiLSTM = BiLSTMModel(input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_prob)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_BiLSTM.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    model_BiLSTM.train()
    epoch_losses = []
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model_BiLSTM(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    avg_loss = sum(epoch_losses) / len(epoch_losses)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')

model_BiLSTM.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = model_BiLSTM(X_batch)
        predicted = (torch.sigmoid(output) >= 0.5).int()
        y_pred.extend(predicted.tolist())
        y_true.extend(y_batch.int().tolist())

y_true_int = np.array(y_true).astype(int)
y_pred_int = np.array(y_pred).astype(int)

accuracy = accuracy_score(y_true_int, y_pred_int)
precision = precision_score(y_true_int, y_pred_int, zero_division=0)
recall = recall_score(y_true_int, y_pred_int, zero_division=0)
f1 = f1_score(y_true_int, y_pred_int, zero_division=0)

print(f'BiLSTM Model - Accuracy: {accuracy:.4f}')
print(f'BiLSTM Model - Precision: {precision:.4f}')
print(f'BiLSTM Model - Recall: {recall:.4f}')
print(f'BiLSTM Model - F1 Score: {f1:.4f}')


Epoch 1/100, Loss: 0.5837
Epoch 2/100, Loss: 0.3988
Epoch 3/100, Loss: 0.3698
Epoch 4/100, Loss: 0.3590
Epoch 5/100, Loss: 0.3515
Epoch 6/100, Loss: 0.3443
Epoch 7/100, Loss: 0.3318
Epoch 8/100, Loss: 0.3266
Epoch 9/100, Loss: 0.3175
Epoch 10/100, Loss: 0.3171
Epoch 11/100, Loss: 0.3091
Epoch 12/100, Loss: 0.2988
Epoch 13/100, Loss: 0.2959
Epoch 14/100, Loss: 0.2894
Epoch 15/100, Loss: 0.2878
Epoch 16/100, Loss: 0.2789
Epoch 17/100, Loss: 0.2696
Epoch 18/100, Loss: 0.2678
Epoch 19/100, Loss: 0.2599
Epoch 20/100, Loss: 0.2560
Epoch 21/100, Loss: 0.2527
Epoch 22/100, Loss: 0.2512
Epoch 23/100, Loss: 0.2488
Epoch 24/100, Loss: 0.2482
Epoch 25/100, Loss: 0.2395
Epoch 26/100, Loss: 0.2366
Epoch 27/100, Loss: 0.2354
Epoch 28/100, Loss: 0.2287
Epoch 29/100, Loss: 0.2263
Epoch 30/100, Loss: 0.2308
Epoch 31/100, Loss: 0.2162
Epoch 32/100, Loss: 0.2177
Epoch 33/100, Loss: 0.2125
Epoch 34/100, Loss: 0.2225
Epoch 35/100, Loss: 0.2163
Epoch 36/100, Loss: 0.2080
Epoch 37/100, Loss: 0.2104
Epoch 38/1

In [35]:
file_path = 'extracted_features-2.csv'
data = pd.read_csv(file_path)

data['trigger'] = data['trigger'].map({-1: 0, 1: 1})

train_data = data[data['participant_id'] != 5]
test_data = data[data['participant_id'] == 5]

X_train = train_data.drop(columns=['trigger', 'participant_id']).values
y_train = train_data['trigger'].values
X_test = test_data.drop(columns=['trigger', 'participant_id']).values
y_test = test_data['trigger'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_prob):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_dim, hidden_dim1, batch_first=True)
        self.dropout1 = nn.Dropout(dropout_prob)
        self.lstm2 = nn.LSTM(hidden_dim1, hidden_dim2, batch_first=True)
        self.dropout2 = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.dropout2(x)
        x = self.fc(x[:, -1, :])
        return x.squeeze()

input_dim = X_train_tensor.shape[2]
hidden_dim1 = 10
hidden_dim2 = 5
output_dim = 1
dropout_prob = 0.2

model_LSTM = LSTMModel(input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_prob)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_LSTM.parameters(), lr=0.001)

epochs = 50
for epoch in range(epochs):
    model_LSTM.train()
    epoch_losses = []
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model_LSTM(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    avg_loss = sum(epoch_losses) / len(epoch_losses)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')

model_LSTM.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = model_LSTM(X_batch)
        predicted = (torch.sigmoid(output) >= 0.5).int()
        y_pred.extend(predicted.tolist())
        y_true.extend(y_batch.int().tolist())


y_true_int = np.array(y_true).astype(int)
y_pred_int = np.array(y_pred).astype(int)
accuracy = accuracy_score(y_true_int, y_pred_int)
precision = precision_score(y_true_int, y_pred_int, zero_division=0)
recall = recall_score(y_true_int, y_pred_int, zero_division=0)
f1 = f1_score(y_true_int, y_pred_int, zero_division=0)

print(f'LSTM Model - Accuracy: {accuracy:.4f}')
print(f'LSTM Model - Precision: {precision:.4f}')
print(f'LSTM Model - Recall: {recall:.4f}')
print(f'LSTM Model - F1 Score: {f1:.4f}')


Epoch 1/50, Loss: 0.6668
Epoch 2/50, Loss: 0.4521
Epoch 3/50, Loss: 0.3905
Epoch 4/50, Loss: 0.3756
Epoch 5/50, Loss: 0.3726
Epoch 6/50, Loss: 0.3665
Epoch 7/50, Loss: 0.3647
Epoch 8/50, Loss: 0.3568
Epoch 9/50, Loss: 0.3547
Epoch 10/50, Loss: 0.3487
Epoch 11/50, Loss: 0.3473
Epoch 12/50, Loss: 0.3442
Epoch 13/50, Loss: 0.3389
Epoch 14/50, Loss: 0.3340
Epoch 15/50, Loss: 0.3354
Epoch 16/50, Loss: 0.3292
Epoch 17/50, Loss: 0.3248
Epoch 18/50, Loss: 0.3206
Epoch 19/50, Loss: 0.3197
Epoch 20/50, Loss: 0.3217
Epoch 21/50, Loss: 0.3146
Epoch 22/50, Loss: 0.3103
Epoch 23/50, Loss: 0.3114
Epoch 24/50, Loss: 0.3098
Epoch 25/50, Loss: 0.3111
Epoch 26/50, Loss: 0.3036
Epoch 27/50, Loss: 0.3090
Epoch 28/50, Loss: 0.3086
Epoch 29/50, Loss: 0.3024
Epoch 30/50, Loss: 0.3032
Epoch 31/50, Loss: 0.2933
Epoch 32/50, Loss: 0.2922
Epoch 33/50, Loss: 0.2918
Epoch 34/50, Loss: 0.2945
Epoch 35/50, Loss: 0.2863
Epoch 36/50, Loss: 0.2912
Epoch 37/50, Loss: 0.2869
Epoch 38/50, Loss: 0.2820
Epoch 39/50, Loss: 0.

In [36]:
file_path = 'extracted_features-2.csv'
data = pd.read_csv(file_path)

data['trigger'] = data['trigger'].map({-1: 0, 1: 1})

train_data = data[data['participant_id'] != 5]
test_data = data[data['participant_id'] == 5]
X_train = train_data.drop(columns=['trigger', 'participant_id']).values
y_train = train_data['trigger'].values
X_test = test_data.drop(columns=['trigger', 'participant_id']).values
y_test = test_data['trigger'].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

class X_LSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(X_LSTMCell, self).__init__()
        self.hidden_dim = hidden_dim
        self.input_gate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.forget_gate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.cell_gate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.output_gate = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.modulation_gate = nn.Linear(input_dim + hidden_dim, hidden_dim)

    def forward(self, x, hidden):
        h_prev, c_prev = hidden
        combined = torch.cat([x, h_prev], dim=1)
        i_t = torch.sigmoid(self.input_gate(combined))
        f_t = torch.sigmoid(self.forget_gate(combined))
        o_t = torch.sigmoid(self.output_gate(combined))
        c_tilde = torch.tanh(self.cell_gate(combined))
        m_t = torch.sigmoid(self.modulation_gate(combined))
        c_t = f_t * c_prev + i_t * c_tilde * m_t
        h_t = o_t * torch.tanh(c_t)
        return h_t, c_t

class X_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout_prob):
        super(X_LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_prob)
        self.cells = nn.ModuleList([X_LSTMCell(input_dim if i == 0 else hidden_dim, hidden_dim) for i in range(num_layers)])

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        h = torch.zeros(batch_size, self.hidden_dim).to(x.device)
        c = torch.zeros(batch_size, self.hidden_dim).to(x.device)

        for layer in range(self.num_layers):
            outputs = []
            for t in range(seq_len):
                h, c = self.cells[layer](x[:, t, :], (h, c))
                outputs.append(h)
            x = torch.stack(outputs, dim=1)
            if layer != self.num_layers - 1:
                x = self.dropout(x)

        return x, (h, c)

class XLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_prob):
        super(XLSTMModel, self).__init__()
        self.xlstm1 = X_LSTM(input_dim, hidden_dim1, num_layers=1, dropout_prob=dropout_prob)
        self.xlstm2 = X_LSTM(hidden_dim1, hidden_dim2, num_layers=1, dropout_prob=dropout_prob)
        self.fc = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x, _ = self.xlstm1(x)
        x, _ = self.xlstm2(x)
        x = self.fc(x[:, -1, :])
        return x.squeeze()

input_dim = X_train_tensor.shape[2]
hidden_dim1 = 10
hidden_dim2 = 5
output_dim = 1
dropout_prob = 0.2

model_xLSTM = XLSTMModel(input_dim, hidden_dim1, hidden_dim2, output_dim, dropout_prob)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_xLSTM.parameters(), lr=0.001)

epochs = 50
for epoch in range(epochs):
    model_xLSTM.train()
    epoch_losses = []
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model_xLSTM(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    avg_loss = sum(epoch_losses) / len(epoch_losses)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}')

model_xLSTM.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = model_xLSTM(X_batch)
        predicted = (torch.sigmoid(output) >= 0.5).int()
        y_pred.extend(predicted.tolist())
        y_true.extend(y_batch.int().tolist())

y_true_int = np.array(y_true).astype(int)
y_pred_int = np.array(y_pred).astype(int)
accuracy = accuracy_score(y_true_int, y_pred_int)
precision = precision_score(y_true_int, y_pred_int, zero_division=0)
recall = recall_score(y_true_int, y_pred_int, zero_division=0)
f1 = f1_score(y_true_int, y_pred_int, zero_division=0)

print(f'XLSTM Model - Accuracy: {accuracy:.4f}')
print(f'XLSTM Model - Precision: {precision:.4f}')
print(f'XLSTM Model - Recall: {recall:.4f}')
print(f'XLSTM Model - F1 Score: {f1:.4f}')


Epoch 1/50, Loss: 0.5578
Epoch 2/50, Loss: 0.4040
Epoch 3/50, Loss: 0.3650
Epoch 4/50, Loss: 0.3566
Epoch 5/50, Loss: 0.3508
Epoch 6/50, Loss: 0.3450
Epoch 7/50, Loss: 0.3397
Epoch 8/50, Loss: 0.3348
Epoch 9/50, Loss: 0.3297
Epoch 10/50, Loss: 0.3250
Epoch 11/50, Loss: 0.3203
Epoch 12/50, Loss: 0.3162
Epoch 13/50, Loss: 0.3112
Epoch 14/50, Loss: 0.3066
Epoch 15/50, Loss: 0.3028
Epoch 16/50, Loss: 0.2976
Epoch 17/50, Loss: 0.2936
Epoch 18/50, Loss: 0.2895
Epoch 19/50, Loss: 0.2843
Epoch 20/50, Loss: 0.2803
Epoch 21/50, Loss: 0.2752
Epoch 22/50, Loss: 0.2714
Epoch 23/50, Loss: 0.2669
Epoch 24/50, Loss: 0.2630
Epoch 25/50, Loss: 0.2593
Epoch 26/50, Loss: 0.2550
Epoch 27/50, Loss: 0.2521
Epoch 28/50, Loss: 0.2475
Epoch 29/50, Loss: 0.2435
Epoch 30/50, Loss: 0.2402
Epoch 31/50, Loss: 0.2358
Epoch 32/50, Loss: 0.2334
Epoch 33/50, Loss: 0.2299
Epoch 34/50, Loss: 0.2255
Epoch 35/50, Loss: 0.2223
Epoch 36/50, Loss: 0.2195
Epoch 37/50, Loss: 0.2174
Epoch 38/50, Loss: 0.2132
Epoch 39/50, Loss: 0.